In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [26]:
file_path = "../drug200.csv"
data = pd.read_csv(file_path)
data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


# Codificacoin de las variables categoricas
Consiste en tomar todas las variables que son cualitativas y transformarlas a valores numericos con el fin de que sea mas facil a nivel matematico su procesamiento

In [27]:
label_encoders = {}
for column in ["Sex", "BP", "Cholesterol", "Drug"]:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Separacion de caracteristicas
Se dvide en dos ejes, el de x en donde estan los valores independientes de la funcion y y en donde esta la variable dependiente (En este caso Drug)

In [28]:
x = data.drop("Drug", axis=1).values
y = data["Drug"].values

# Normalizacion
Se normalizan con el fin de que la media sea cero y la desviacion estandar uno, esto permite que la red neuronal entrene de forma mas eficiente, ya que estas aprenden mejor cuando las entradas se encuentran en rangos similares, haciendo que las que tienen valores mas grandes no dominen el proceso de aprendizaje y de esa forma prestar atencion a todo los detalles por mas minimos que sean

In [29]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Division de datos en conjuntos
Se hace un conjunto de entrenamiento, que consiste en el 80% y uno de priebas que consiste en el 20%, es decir que el 80% de todo el dataset se usa para entrenar mientras que el otro 20 para realizar pruebas

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

# Red neuronal
Se basa en el uso de ReLu y softmax, pero para eso primero hay que explicar las partes basicas

## Pesos
Los coeficientes que se van ajustando durante el entrenamiento de la red neuronal,cada conexion tiene un peso que determina la importancia de su senal

## Sesgo
Termino adicional que se le agrega a la entrada de cada neurona para ajustar la salida de la funcino de activacion, se suele inciializar con ceros

## ReLU
Permite introducir no linealidad a la red
ReLU(z) = max(0, z)

## Softmax
Convierte las salidas en probabilidades, esto permite clasificacion multiclase
softmax(z_i) = exp(z_i) / sum(exp(z_j))

## Forward pass
Es la forma en la que la entrada se propaga a traves de la red a traves de todas las capaz hasta obteer la salida, en donde:

### Capa de entrada oculta:
Z1 =X⋅W 1​ +b 1
​A1 = ReLU(Z1)

### Capa oculta a capa de salida
Z2 = A1 * W2 + b2
A2 = softmax(Z2)

## Backward pass
Tiene el fin de calcular los pesos y sesgos para posteriormente ajustarlos e ir mejorando la precision del modelo

### Cuando hay error en la capa de salida:
dZ2 = A2 - Yone-hot
dW2 = 1/m * (A_{1}^{t} * dZ2)
dv2 = 1/m * sumatoria(dZ2)

### Cuando hay error en la capa oculta:
dA1 = dZ2 * W_{2}^{T}
dZ1 = dA1 * ReLU'(Z1)
dW1 = 1/m * (X^T * dZ1)
db1 = 1/m * sumatoria(dZ2)

## Actualizar los pesos y sesgos
W1 = W1 - alpha * dW1
b1 = b1 - alpha * db1
W2 = W2 - alpha * dW2
b2 = b2 - alpha * db2

## Entrenamiento
Recibe los datos de entradas, los resultados de y y los ephocs (Numero de veces que el modelo vera todo el conjunto de los datos), por cada vez que se ven los datos la red hace un forward pass para calcular las salidas de la red neuronal para las entradas dadas, lo que pasa las entradas a traves de las capaz de cada red aplicando sus funciones de activacion, posteriormente hace un backward pass en donde se calculan los pesos usando el error que hubo entre las predicciones y los valores reales, a partir de ello actualiza los pesos

In [31]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Inicializar pesos
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
    
    def relu(self, Z):
        return np.maximum(0, Z)
    
    def relu_derivative(self, Z):
        return Z > 0
    
    def softmax(self, Z):
        exp_values = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)
    
    def forward(self, X):
        # Capa de entrada a capa oculta
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        
        # Capa oculta a capa de salida
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.softmax(self.Z2)
        
        return self.A2
    
    def backward(self, X, y, output):
        # Convertir y a one-hot encoding
        y_one_hot = np.zeros((y.size, self.output_size))
        y_one_hot[np.arange(y.size), y] = 1
        
        # Error en la capa de salida
        dZ2 = output - y_one_hot
        dW2 = np.dot(self.A1.T, dZ2) / y.size
        db2 = np.sum(dZ2, axis=0, keepdims=True) / y.size
        
        # Error en la capa oculta
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / y.size
        db1 = np.sum(dZ1, axis=0, keepdims=True) / y.size
        
        # Actualizar pesos y sesgos
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
    
    def train(self, X, y, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
    
    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

In [32]:
input_size = x_train.shape[1]
hidden_size = 30000
output_size = len(np.unique(y))
learning_rate = 0.01

nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)

nn.train(x_train, y_train, 1000)

y_pred = nn.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


In [33]:
def predict_drug(age, sex, bp, cholesterol, na_to_k_ratio):
    input_data = pd.DataFrame({
        "Age": [age],
        "Sex": [sex],
        "BP": [bp],
        "Cholesterol": [cholesterol],
        "Na_to_K": [na_to_k_ratio]
    })
    
    for column in ["Sex", "BP", "Cholesterol"]:
        le = label_encoders[column]
        input_data[column] = le.transform(input_data[column])
        
    input_features = scaler.transform(input_data.values)
    
    prediction = nn.predict(input_features)
    predicted_drug = label_encoders["Drug"].inverse_transform(prediction)
    
    return predicted_drug[0]

In [34]:
new_patient_prediction = predict_drug(52, "M", "HIGH", "HIGH", 12)
print(f"Predicted drug: {new_patient_prediction}")

Predicted drug: drugA


In [35]:
import joblib

joblib.dump(nn, 'neural_network_model.pkl')
joblib.dump((label_encoders, scaler, x_test, y_test), 'preprocessing.pkl')

['preprocessing.pkl']